# Husky2
Em teoria é melhor
- `Forms` melhorados
- Mais indicadores

In [314]:
import pandas as pd
#import xlwings as xw
import openpyxl as op
import numpy as np
import datetime as dt
import numpy as np
import os
import math

print('Neste modelos vamos fazer uma experiência e não usar Odds')
print('Em vex disso, usamos os xPTS para calcular o "power" de uma equipa')

Neste modelos vamos fazer uma experiência e não usar Odds
Em vex disso, usamos os xPTS para calcular o "power" de uma equipa


In [315]:
#Muito Importante
Teams={'Prem':20,'LL':20,'L':20,'SA':20,'LP':18,'Bund':18,'Ered':18}

In [316]:
pd.set_option('display.max_rows',2000)
pd.set_option('display.max_columns',1000)

# Funções auxiliares
Algumes vêm no python Notebook

`new_find_data()` o find_data, para os ficheiros New

In [317]:
def new_find_data(os_path):
    lista=os.listdir(os_path)
    data_newfiles=[]
    for i in lista:
        if (i[-3:]=='___') and (i[:3].lower()=='new'):
            data_newfiles+=[i]
    print(data_newfiles)
    return data_newfiles

In [318]:
def season_index(path,season): #damos o path e a época e devolve o índice do listdir
    seas=str(season)
    seas=seas[-2:]
    lista=os.listdir(path)
    i=0
    found=False
    while i<len(lista) and not(found):
        if lista[i][-7:-5]==seas:
            found=True
        else:
            i+=1
    if not(found):
        print('Esta season não existe')
        raise('Meh')
    return i

`round_down` arredonda (para os números decimais que quisermos) para baixo

In [319]:
def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

In [320]:
round_down(1.39,1)

1.3

In [321]:
def check_wrongdate(file,sheet=0):
    #file=pd.read_excel(path+'.xlsx',sheet)
    limit=len(file.index)
    over_month=0
    month2=0
    counter=0
    counter_pd=0
    for i in range(1,limit): #ao começar no 1, temos sempre um valor anterior para comparar, se começasse no zero, comparava com o último valor [-1]
        day_=file.iloc[i]['Date']
        day2=file.iloc[i-1]['Date']
        if isinstance(day_,dt.datetime):
            counter+=1
            if isinstance(day2,str):
                if len(day2)==10:
                    date_str=dt.datetime.strptime(day2,"%d/%m/%Y")
                elif len(day2)==8:                                        #serve para datas qnão têm '20' (2/3/15)
                    day2=day2[:6]+"20"+day2[6:]
                    date_str=dt.datetime.strptime(day2,"%d/%m/%Y")
                else: 
                    print("Oops! Algo está a falhar na função date, qnd a data é uma string!")
                if ((day_.month)-(date_str.month)!=1) and ((day_.month)-(date_str.month)!=0):
                    over_month=1 #qnd muda de str para datetime, e estão trocados mes e dia a diferença entre os meses é estranha (diferente de 1)
            elif isinstance(day2,dt.datetime):
                if (day2.day==day_.day) and (day2.month!=day_.month):
                    #este if é para 'apanhar' qnd troca mes com dia, então o dia é igual, mas o mês muda em linhas consecutivas
                    month2=1
        elif isinstance(day_,pd.Timestamp):
            counter_pd+=1
    if (over_month+month2)==2:
        return (True, 
               str(counter)+' Datas em Datetime',
               str(counter_pd)+' Datas em Timestamp')
    else:
        return (False, 
               str(counter)+' Datas em Datetime',
               str(counter_pd)+' Datas em Timestamp',
               'over_month = '+str(over_month),
               'month2 = '+str(month2))

def swap_datetime(date):
    return dt.datetime.strptime((dt.datetime.strftime(date,'%Y/%d/%m')),'%Y/%m/%d')

def all_datetime(path,sheet=0,write=True): #se write=True, escreve o ficheiro nos Documentos do PC
    #false, dá-nos apenas o ficheiro já tratado aqui no python
    file=pd.read_excel(path+'.xlsx',sheet)
    limit=len(file.index)
    for it in range(0,limit):    
        if isinstance(file.iloc[it]['Date'],dt.datetime):
            file.at[it,'Date']=swap_datetime(file.iloc[it]['Date'])
        elif isinstance(file.iloc[it]['Date'],str):
            d=file.iloc[it]['Date']
            if len(d)==8:
                d=d[:6]+"20"+d[6:]
            file.at[it,'Date']=dt.datetime.strptime(d,"%d/%m/%Y")
    if write:
        with pd.ExcelWriter(path+".xlsx") as writer:
            file.to_excel(writer, 'Sheet1', index=None)
            writer.save()
    else:
        return file

In [322]:
def findall_clubs(path,teams):
    import pandas as pd
    file=pd.read_excel(path+'.xlsx')
    lista_c=[]
    counter=0
    i=0
    done=False
    while i<len(file.index) and not(done):
        if i==teams+50:
            print('Está a ver demasiadas linhas, tem erro')
        if file.iloc[i]['HomeTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['HomeTeam']]
            counter+=1
        if file.iloc[i]['AwayTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['AwayTeam']]
            counter+=1
        if counter==teams:
            done=True
        i+=1
    return lista_c 

In [323]:
def findall_clubs2_new(path): #menos eficiente, mas não precisa do número das equipas
    import pandas as pd
    #import xlwings
    #xwfile=xw.Book(path+".xlsx") #assim não dá permission error
    #xwfile.close()
    file=pd.read_excel(path+'.xlsx')
    lista_c=[]
    for i in range(0,len(file)):
        H=file.iloc[i]['HT']
        A=file.iloc[i]['AT']
        if H not in lista_c:
            lista_c+=[H]
        if A not in lista_c:
            lista_c+=[A]
            
    return lista_c
    
    

# ALPHA Functions

In [324]:
def form1(df,clubs,x,diff=True): #usa xG, model 5, Ean/Eadv
    if x!=5:
        print('We gotta redo this fuction maman')
        print('Cause x != 5 :(')
        raise('Ja foste')
    for club in clubs:
        vars()[club]=[[],[],0] # 0 xG // 1 Form per game
    H_form=[]
    A_form=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_form+=[0]
        else:
            H_form+=[
                round((round((vars()[H][1][-5])*0.1,2))+(round((vars()[H][1][-4])*0.15,2))+(round((vars()[H][1][-3])*0.2,2))+
                (round((vars()[H][1][-2])*0.25,2))+(round((vars()[H][1][-1])*0.3,2)),1)
            ]
            
        if vars()[A][-1]<x:
            A_form+=[0]
        else:
            A_form+=[
                round((round((vars()[A][1][-5])*0.1,2))+(round((vars()[A][1][-4])*0.15,2))+(round((vars()[A][1][-3])*0.2,2))+
                      (round((vars()[A][1][-2])*0.25,2))+(round((vars()[A][1][-1])*0.3,2)),1)
            ]
        #  # #  # # # # # # #
        vars()[H][0]+=[df.iloc[i]['HxG']]
        vars()[A][0]+=[df.iloc[i]['AxG']]
        
        if vars()[H][-1]<x:
            vars()[H][1]+=[
                (((sum(vars()[H][0]))/(sum(vars()[A][0])))*(vars()[H][0][-1]))
            ]
        elif vars()[H][-1]>=x:
            vars()[H][1]+=[
                (((sum(vars()[H][0][-x:]))/(sum(vars()[A][0][-x:])))*(vars()[H][0][-1]))
            ]
        else:
            raise('Meh')
            
            
        if vars()[A][-1]<x:
            vars()[A][1]+=[
                (((sum(vars()[A][0]))/(sum(vars()[H][0])))*(vars()[A][0][-1]))
            ]
        elif vars()[A][-1]>=x:
            vars()[A][1]+=[
                (((sum(vars()[A][0][-x:]))/(sum(vars()[H][0][-x:])))*(vars()[A][0][-1]))
            ]
        else:
            raise('Meh2')
        
        
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
            
    form_diff=list(map(lambda x,y:x-y,H_form,A_form))
    if diff:
        output=pd.DataFrame({'Form1_Diff'+str(x):form_diff})
    else:
        output=pd.DataFrame({'Form1_H'+str(x):H_form,
                            'Form1_A'+str(x):A_form,
                            'Form1_Diff'+str(x):form_diff})
    return output
        
        
        

In [325]:
def form2(df,clubs,x,diff=True): #usa xPTS, model 4 Eadv/Ean
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],0] #0 xPTS / Form per Game
    H_form2=[]
    A_form2=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        if vars()[H][-1]<x:
            H_form2+=[0]
        else:
            H_form2+=[
                round(((vars()[H][1][-4])*0.15)+((vars()[H][1][-3])*0.2)+
                      ((vars()[H][1][-2])*0.3)+((vars()[H][1][-1])*0.35),1)
            ]
            
        if vars()[A][-1]<x:
            A_form2+=[0]
        else:
            A_form2+=[
                round(((vars()[A][1][-4])*0.15)+((vars()[A][1][-3])*0.2)+
                      ((vars()[A][1][-2])*0.3)+((vars()[A][1][-1])*0.35),1)
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HxPTS']]
        vars()[A][0]+=[df.iloc[i]['AxPTS']]
        if vars()[H][-1]==0:
            vars()[H][1]+=[0]
        
        elif 0<vars()[H][-1]<x:
            if sum(vars()[H][0][:-1])==0 and H=='Alaves': #Alaves em 2016 Vs Atleti Mad. teve xPTS=0.0 (e ficou 1-1 xD)
                print('Le Lord Alaves Vs Atleti 2016 (1-1 xD)')
                vars()[H][1]+=[
                    (((sum(vars()[A][0][:-1]))/(0.01))*(vars()[H][0][-1])) 
                ]
            elif sum(vars()[H][0][:-1])==0 and H=='Udinese': #Udinese também tem um jogo com xPTS=0.0 (perdeu 4-0)
                print('Roma Vs Lord Udinese 2017 (4-0)')
                vars()[H][1]+=[
                    (((sum(vars()[A][0][:-1]))/(0.01))*(vars()[H][0][-1])) 
                ]
            else:
                vars()[H][1]+=[
                    (((sum(vars()[A][0][:-1]))/(sum(vars()[H][0][:-1])))*(vars()[H][0][-1])) 
                ]
        else:
            if sum(vars()[H][0][-x:-1])==0:
                print(H,A,'2')
            vars()[H][1]+=[
                (((sum(vars()[A][0][-x:-1]))/(sum(vars()[H][0][-x:-1])))*(vars()[H][0][-1]))
            ]
        
        if vars()[A][-1]==0:
            vars()[A][1]+=[0]
        elif 0<vars()[A][-1]<x:
            if sum(vars()[A][0][:-1])==0:
                print(H,A,'3')
            vars()[A][1]+=[
                (((sum(vars()[H][0][:-1]))/(sum(vars()[A][0][:-1])))*(vars()[A][0][-1]))
            ]
        else: 
            if sum(vars()[A][0][-x:-1])==0:
                print(H,A,'4')
            vars()[A][1]+=[
                (((sum(vars()[H][0][-x:-1]))/(sum(vars()[A][0][-x:-1])))*(vars()[A][0][-1]))
            ]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # form2 diff H-A
    form2_diff=list(map(lambda x,y:x-y,H_form2,A_form2))
    
    if diff:
        output=pd.DataFrame({'Form2_Diff'+str(x):form2_diff})
    else:
        output=pd.DataFrame({'Form2_H'+str(x):H_form2,
                            'Form2_A'+str(x):A_form2,
                            'Form2_Diff'+str(x):form2_diff})
        
    return output


In [326]:
def form3(df,clubs,x,diff=True): #usa xPTS, model 4, Ean/Eadv
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],0] #0 xPTS / Form per Game
    H_form2=[]
    A_form2=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_form2+=[0]
        else:
            H_form2+=[
                round(((vars()[H][1][-4])*0.15)+((vars()[H][1][-3])*0.2)+
                      ((vars()[H][1][-2])*0.3)+((vars()[H][1][-1])*0.35),1)
            ]
            
        if vars()[A][-1]<x:
            A_form2+=[0]
        else:
            A_form2+=[
                round(((vars()[A][1][-4])*0.15)+((vars()[A][1][-3])*0.2)+
                      ((vars()[A][1][-2])*0.3)+((vars()[A][1][-1])*0.35),1)
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HxPTS']]
        vars()[A][0]+=[df.iloc[i]['AxPTS']]
        if vars()[H][-1]==0:
            vars()[H][1]+=[0]
        
        elif 0<vars()[H][-1]<x:
            if sum(vars()[A][0][:-1]) == 0:
                vars()[H][1]+=[
                    (((sum(vars()[H][0][:-1]))/((df.iloc[i]['AxPTS'])*len(vars()[H][0][:-1])))*(vars()[H][0][-1])) 
                ]
            else:
                vars()[H][1]+=[
                    (((sum(vars()[H][0][:-1]))/(sum(vars()[A][0][:-1])))*(vars()[H][0][-1])) 
                ]
        else:
            if sum(vars()[A][0][-x:-1]) == 0:
                vars()[H][1]+=[
                    (((sum(vars()[H][0][-x:-1]))/((df.iloc[i]['AxPTS'])*x))*(vars()[H][0][-1]))
                ]
            else:
                vars()[H][1]+=[
                    (((sum(vars()[H][0][-x:-1]))/(sum(vars()[A][0][-x:-1])))*(vars()[H][0][-1]))
                ]
                
        #### #  # # #  # # # #  # # #  # # # #  # # #  # # #  # 
        if vars()[A][-1]==0:
            vars()[A][1]+=[0]
        elif 0<vars()[A][-1]<x:
            if sum(vars()[H][0][:-1]) == 0:
                vars()[A][1]+=[
                    (((sum(vars()[A][0][:-1]))/((df.iloc[i]['HxPTS'])*len(vars()[A][0][:-1])))*(vars()[A][0][-1]))
                ]
            else:
                vars()[A][1]+=[
                    (((sum(vars()[A][0][:-1]))/(sum(vars()[H][0][:-1])))*(vars()[A][0][-1]))
                ]
        else: 
            if sum(vars()[H][0][-x:-1]) == 0:
                vars()[A][1]+=[
                    (((sum(vars()[A][0][-x:-1]))/((df.iloc[i]['HxPTS'])*x))*(vars()[A][0][-1]))
                ]
            else:
                vars()[A][1]+=[
                    (((sum(vars()[A][0][-x:-1]))/(sum(vars()[H][0][-x:-1])))*(vars()[A][0][-1]))
                ]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # form2 diff H-A
    form2_diff=list(map(lambda x,y:x-y,H_form2,A_form2))
    
    if diff:
        output=pd.DataFrame({'Form3_Diff'+str(x):form2_diff})
    else:
        output=pd.DataFrame({'Form3_H'+str(x):H_form2,
                            'Form3_A'+str(x):A_form2,
                            'Form3_Diff'+str(x):form2_diff})
        
    return output

In [327]:
def new_form(df,clubs,x,diff=True):
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],0] #0 xPTS / Form per Game
    H_newform=[]
    A_newform=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_newform+=[0]
        else:
            H_newform+=[
                round(((vars()[H][1][-4])*0.15)+((vars()[H][1][-3])*0.2)+
                      ((vars()[H][1][-2])*0.3)+((vars()[H][1][-1])*0.35),1)
            ]
            
        if vars()[A][-1]<x:
            A_newform+=[0]
        else:
            A_newform+=[
                round(((vars()[A][1][-4])*0.15)+((vars()[A][1][-3])*0.2)+
                      ((vars()[A][1][-2])*0.3)+((vars()[A][1][-1])*0.35),1)
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HxPTS']]
        vars()[A][0]+=[df.iloc[i]['AxPTS']]
        if vars()[H][-1]==0:
            vars()[H][1]+=[0]
        
        elif 0<vars()[H][-1]<x:
            vars()[H][1]+=[
                (sum(vars()[A][0][:-1]))*(vars()[H][0][-1]) 
            ]
        else:
            vars()[H][1]+=[
                (sum(vars()[A][0][-x:-1]))*(vars()[H][0][-1])
            ]
        
        if vars()[A][-1]==0:
            vars()[A][1]+=[0]
        elif 0<vars()[A][-1]<x:
            vars()[A][1]+=[
                (sum(vars()[H][0][:-1]))*(vars()[A][0][-1])
            ]
        else: 
            vars()[A][1]+=[
                (sum(vars()[H][0][-x:-1]))*(vars()[A][0][-1])
            ]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    newform_diff=list(map(lambda x,y:x-y,H_newform,A_newform))
    
    if diff:
        output=pd.DataFrame({'NewForm_Diff'+str(x):newform_diff})
    else:
        output=pd.DataFrame({'NewForm_H'+str(x):H_newform,
                            'NewForm_A'+str(x):A_newform,
                            'NewForm_Diff'+str(x):newform_diff})
        
    return output

In [328]:
def Def_form(df,clubs,x,diff=True): #pertence ao grupo do xConcd
    #usa o método new_form
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 xC / 1 xG / 2 Form per Game
    H_Def=[]
    A_Def=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_Def+=[0]
        else:
            H_Def+=[
                round(100*(((vars()[H][2][-4])*0.15)+((vars()[H][2][-3])*0.2)+
                      ((vars()[H][2][-2])*0.3)+((vars()[H][2][-1])*0.35)))
            ]
            
        if vars()[A][-1]<x:
            A_Def+=[0]
        else:
            A_Def+=[
                round(100*(((vars()[A][2][-4])*0.15)+((vars()[A][2][-3])*0.2)+
                      ((vars()[A][2][-2])*0.3)+((vars()[A][2][-1])*0.35)))
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['AxG']] #xConcd
        vars()[A][0]+=[df.iloc[i]['HxG']] #xConcd
        vars()[H][1]+=[df.iloc[i]['HxG']] #xG
        vars()[A][1]+=[df.iloc[i]['AxG']] #xG
        if vars()[H][-1]==0:
            vars()[H][2]+=[0]
        
        elif 0<vars()[H][-1]<x:
            try:
                vars()[H][2]+=[
                    (sum(vars()[A][1][:-1])/len(vars()[A][1][:-1]))-(vars()[H][0][-1])
                ]
            except:
                print(A,'Away, Def_Form, I')
                vars()[H][2]+=[0]
        else:
            try:
                vars()[H][2]+=[
                    (sum(vars()[A][1][-x:-1])/len(vars()[A][1][-x:-1]))-(vars()[H][0][-1])
                ]
            except:
                print(A,'Away, Def_Form, II')
                vars()[H][2]+=[0]
        
        if vars()[A][-1]==0:
            vars()[A][2]+=[0]
        elif 0<vars()[A][-1]<x:
            try:
                vars()[A][2]+=[
                    (sum(vars()[H][1][:-1])/len(vars()[H][1][:-1]))-(vars()[A][0][-1])
                ]
            except:
                print(H,'HomeTeam, Def_form, I')
                vars()[A][2]+=[0] #se não houver info dizemos q é neutro (0)
        else: 
            try:
                vars()[A][2]+=[
                    (sum(vars()[H][1][-x:-1])/len(vars()[H][1][-x:-1]))-(vars()[A][0][-1])
                ]
            except:
                print(H,'HomeTeam, Def_form, II')
                vars()[A][2]+=[0]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    def_form_diff=list(map(lambda x,y:x-y,H_Def,A_Def))
    
    if diff:
        output=pd.DataFrame({'DeForm_Diff'+str(x):def_form_diff})
    else:
        output=pd.DataFrame({'DeForm_H'+str(x):H_Def,
                            'DeForm_A'+str(x):A_Def,
                            'DeForm_Diff'+str(x):def_form_diff})
        
    return output
        

In [329]:
def Atack_form(df,clubs,x,diff=True): #pertence ao grupo do xG
    #usa o método new_form
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 xG / 1 xC / 2 Form per Game
    H_At=[]
    A_At=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_At+=[0]
        else:
            H_At+=[
                round(100*(((vars()[H][2][-4])*0.15)+((vars()[H][2][-3])*0.2)+
                      ((vars()[H][2][-2])*0.3)+((vars()[H][2][-1])*0.35)))
            ]
            
        if vars()[A][-1]<x:
            A_At+=[0]
        else:
            A_At+=[
                round(100*(((vars()[A][2][-4])*0.15)+((vars()[A][2][-3])*0.2)+
                      ((vars()[A][2][-2])*0.3)+((vars()[A][2][-1])*0.35)))
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HxG']] #xG
        vars()[A][0]+=[df.iloc[i]['AxG']] #xG
        vars()[H][1]+=[df.iloc[i]['AxG']] #xConcd
        vars()[A][1]+=[df.iloc[i]['HxG']] #xConcd
        if vars()[H][-1]==0:
            vars()[H][2]+=[0]
        
        elif 0<vars()[H][-1]<x:
            try:
                vars()[H][2]+=[
                    (-(sum(vars()[A][1][:-1])/len(vars()[A][1][:-1])))+(vars()[H][0][-1])
                ]
            except:
                vars()[H][2]+=[0]
        else:
            try:
                vars()[H][2]+=[
                    (-(sum(vars()[A][1][-x:-1])/len(vars()[A][1][-x:-1])))+(vars()[H][0][-1])
                ]
            except:
                vars()[H][2]+=[0]
        
        if vars()[A][-1]==0:
            vars()[A][2]+=[0]
        elif 0<vars()[A][-1]<x:
            try:
                vars()[A][2]+=[
                    (-(sum(vars()[H][1][:-1])/len(vars()[H][1][:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
        else: 
            try:
                vars()[A][2]+=[
                    (-(sum(vars()[H][1][-x:-1])/len(vars()[H][1][-x:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    atk_form_diff=list(map(lambda x,y:x-y,H_At,A_At))
    
    if diff:
        output=pd.DataFrame({'AtkForm_Diff'+str(x):atk_form_diff})
    else:
        output=pd.DataFrame({'AtkForm_H'+str(x):H_At,
                            'AtkForm_A'+str(x):A_At,
                            'AtkForm_Diff'+str(x):atk_form_diff})
        
    return output

In [330]:
def ST_form(df,clubs,x,diff=True):
    #usa o método new_form
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 ShotsT / 1 ST reverse / Form per Game
    H_ST=[]
    A_ST=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_ST+=[0]
        else:
            H_ST+=[
                round(((vars()[H][2][-4])*0.15)+((vars()[H][2][-3])*0.2)+
                      ((vars()[H][2][-2])*0.3)+((vars()[H][2][-1])*0.35))
            ]
            
        if vars()[A][-1]<x:
            A_ST+=[0]
        else:
            A_ST+=[
                round(((vars()[A][2][-4])*0.15)+((vars()[A][2][-3])*0.2)+
                      ((vars()[A][2][-2])*0.3)+((vars()[A][2][-1])*0.35))
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HST']]
        vars()[A][0]+=[df.iloc[i]['AST']]
        vars()[H][1]+=[df.iloc[i]['AST']]
        vars()[A][1]+=[df.iloc[i]['HST']]
        if vars()[H][-1]==0:
            vars()[H][2]+=[0]
        
        elif 0<vars()[H][-1]<x:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][:-1])))+(vars()[H][0][-1]) 
                ]
            except:
                vars()[H][2]+=[0]
        else:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][-x:-1])))+(vars()[H][0][-1])
                ]
            except:
                vars()[H][2]+=[0]
        
        if vars()[A][-1]==0:
            vars()[A][2]+=[0]
        elif 0<vars()[A][-1]<x:
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
        else: 
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][-x:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    st_form_diff=list(map(lambda x,y:x-y,H_ST,A_ST))
    
    if diff:
        output=pd.DataFrame({'STForm_Diff'+str(x):st_form_diff})
    else:
        output=pd.DataFrame({'STForm_H'+str(x):H_ST,
                            'STForm_A'+str(x):A_ST,
                            'STForm_Diff'+str(x):st_form_diff})
        
    return output

In [331]:
def CCC_form(df,clubs,x,diff=True):
    #usa o método do new_form
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 CCC / 1 xPTS / Form per Game
    H_CCC=[]
    A_CCC=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_CCC+=[0]
        else:
            H_CCC+=[
                round(((vars()[H][2][-4])*0.15)+((vars()[H][2][-3])*0.2)+
                      ((vars()[H][2][-2])*0.3)+((vars()[H][2][-1])*0.35))
            ]
            
        if vars()[A][-1]<x:
            A_CCC+=[0]
        else:
            A_CCC+=[
                round(((vars()[A][2][-4])*0.15)+((vars()[A][2][-3])*0.2)+
                      ((vars()[A][2][-2])*0.3)+((vars()[A][2][-1])*0.35))
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HCCC']]
        vars()[A][0]+=[df.iloc[i]['ACCC']]
        vars()[H][1]+=[df.iloc[i]['ACCC']]
        vars()[A][1]+=[df.iloc[i]['HCCC']]
        if vars()[H][-1]==0:
            vars()[H][2]+=[0]
        
        elif 0<vars()[H][-1]<x:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][:-1])))+(vars()[H][0][-1]) 
                ]
            except:
                vars()[H][2]+=[0]
        else:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][-x:-1])))+(vars()[H][0][-1])
                ]
            except:
                vars()[H][2]+=[0]
        
        if vars()[A][-1]==0:
            vars()[A][2]+=[0]
        elif 0<vars()[A][-1]<x:
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
        else: 
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][-x:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    cccform_diff=list(map(lambda x,y:x-y,H_CCC,A_CCC))
    
    if diff:
        output=pd.DataFrame({'CCCForm_Diff'+str(x):cccform_diff})
    else:
        output=pd.DataFrame({'CCCForm_H'+str(x):H_CCC,
                            'CCCForm_A'+str(x):A_CCC,
                            'CCCForm_Diff'+str(x):cccform_diff})
        
    return output

In [332]:
def DEEP_form(df,clubs,x,diff=True):
    #usa o método do new_form
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 DEEP / 1 DEEP reverse / Form per Game
    H_DEEP=[]
    A_DEEP=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_DEEP+=[0]
        else:
            H_DEEP+=[
                round(((vars()[H][2][-4])*0.15)+((vars()[H][2][-3])*0.2)+
                      ((vars()[H][2][-2])*0.3)+((vars()[H][2][-1])*0.35))
            ]
            
        if vars()[A][-1]<x:
            A_DEEP+=[0]
        else:
            A_DEEP+=[
                round(((vars()[A][2][-4])*0.15)+((vars()[A][2][-3])*0.2)+
                      ((vars()[A][2][-2])*0.3)+((vars()[A][2][-1])*0.35))
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HDEEP']]
        vars()[A][0]+=[df.iloc[i]['ADEEP']]
        vars()[H][1]+=[df.iloc[i]['ADEEP']]
        vars()[A][1]+=[df.iloc[i]['HDEEP']]
        if vars()[H][-1]==0:
            vars()[H][2]+=[0]
        
        elif 0<vars()[H][-1]<x:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][:-1])))+(vars()[H][0][-1]) 
                ]
            except:
                vars()[H][2]+=[0]
        else:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][-x:-1])))+(vars()[H][0][-1])
                ]
            except:
                vars()[H][2]+=[0]
        
        if vars()[A][-1]==0:
            vars()[A][2]+=[0]
        elif 0<vars()[A][-1]<x:
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
        else: 
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][-x:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    deepform_diff=list(map(lambda x,y:x-y,H_DEEP,A_DEEP))
    
    if diff:
        output=pd.DataFrame({'DEEPForm_Diff'+str(x):deepform_diff})
    else:
        output=pd.DataFrame({'DEEPForm_H'+str(x):H_DEEP,
                            'DEEPForm_A'+str(x):A_DEEP,
                            'DEEPForm_Diff'+str(x):deepform_diff})
        
    return output

In [333]:
def PPDA_form(df,clubs,x,diff=True):
    #usa o método do new_form
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 PPDA / 1 PPDA reverse / Form per Game
    H_PPDA=[]
    A_PPDA=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_PPDA+=[0]
        else:
            H_PPDA+=[
                round(((vars()[H][2][-4])*0.15)+((vars()[H][2][-3])*0.2)+
                      ((vars()[H][2][-2])*0.3)+((vars()[H][2][-1])*0.35))
            ]
            
        if vars()[A][-1]<x:
            A_PPDA+=[0]
        else:
            A_PPDA+=[
                round(((vars()[A][2][-4])*0.15)+((vars()[A][2][-3])*0.2)+
                      ((vars()[A][2][-2])*0.3)+((vars()[A][2][-1])*0.35))
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HPPDA']]
        vars()[A][0]+=[df.iloc[i]['APPDA']]
        vars()[H][1]+=[df.iloc[i]['APPDA']]
        vars()[A][1]+=[df.iloc[i]['HPPDA']]
        if vars()[H][-1]==0:
            vars()[H][2]+=[0]
        
        elif 0<vars()[H][-1]<x:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][:-1])))+(vars()[H][0][-1]) 
                ]
            except:
                vars()[H][2]+=[0]
        else:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][-x:-1])))+(vars()[H][0][-1])
                ]
            except:
                vars()[H][2]+=[0]
        
        if vars()[A][-1]==0:
            vars()[A][2]+=[0]
        elif 0<vars()[A][-1]<x:
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
        else: 
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][-x:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    ppdaform_diff=list(map(lambda x,y:x-y,H_PPDA,A_PPDA))
    
    if diff:
        output=pd.DataFrame({'PPDAForm_Diff'+str(x):ppdaform_diff})
    else:
        output=pd.DataFrame({'PPDAForm_H'+str(x):H_PPDA,
                            'PPDAForm_A'+str(x):A_PPDA,
                            'PPDAForm_Diff'+str(x):ppdaform_diff})
        
    return output

In [334]:
def Pass_form(df,clubs,x,diff=True):
    #usa o método do new_form
    if x!=5:
        print('Maman, o x tem de ser 5, caso contrario, muda a função')
        raise('Meh')
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 Pass / 1 Pass reverse / Form per Game
    H_Pass=[]
    A_Pass=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_Pass+=[0]
        else:
            H_Pass+=[
                round(((vars()[H][2][-4])*0.15)+((vars()[H][2][-3])*0.2)+
                      ((vars()[H][2][-2])*0.3)+((vars()[H][2][-1])*0.35))
            ]
            
        if vars()[A][-1]<x:
            A_Pass+=[0]
        else:
            A_Pass+=[
                round(((vars()[A][2][-4])*0.15)+((vars()[A][2][-3])*0.2)+
                      ((vars()[A][2][-2])*0.3)+((vars()[A][2][-1])*0.35))
            ]
            
        # # # # # # #  # # # #
        vars()[H][0]+=[df.iloc[i]['HPass']]
        vars()[A][0]+=[df.iloc[i]['APass']]
        vars()[H][1]+=[df.iloc[i]['APass']]
        vars()[A][1]+=[df.iloc[i]['HPass']]
        if vars()[H][-1]==0:
            vars()[H][2]+=[0]
        
        elif 0<vars()[H][-1]<x:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][:-1])))+(vars()[H][0][-1]) 
                ]
            except:
                vars()[H][2]+=[0]
        else:
            try:
                vars()[H][2]+=[
                    (-(np.mean(vars()[A][1][-x:-1])))+(vars()[H][0][-1])
                ]
            except:
                vars()[H][2]+=[0]
        
        if vars()[A][-1]==0:
            vars()[A][2]+=[0]
        elif 0<vars()[A][-1]<x:
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
        else: 
            try:
                vars()[A][2]+=[
                    (-(np.mean(vars()[H][1][-x:-1])))+(vars()[A][0][-1])
                ]
            except:
                vars()[A][2]+=[0]
                
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # # newform diff H-A
    passform_diff=list(map(lambda x,y:x-y,H_Pass,A_Pass))
    
    if diff:
        output=pd.DataFrame({'PassForm_Diff'+str(x):passform_diff})
    else:
        output=pd.DataFrame({'PassForm_H'+str(x):H_Pass,
                            'PassForm_A'+str(x):A_Pass,
                            'PassForm_Diff'+str(x):passform_diff})
        
    return output

In [335]:
def unlucky(df,clubs,x,diff=True):
    #if x!=5:
    #    print('Maman, o x tem de ser 5, caso contrario, muda a função')
    #    raise('Meh')
    for club in clubs:
        vars()[club]=[[],0] 
    H_unlucky=[]
    A_unlucky=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        if vars()[H][-1]<x:
            H_unlucky+=[0]
        else:
            H_unlucky+=[
                round(sum(vars()[H][0][-x:]),1)
            ]
            
        if vars()[A][-1]<x:
            A_unlucky+=[0]
        else:
            A_unlucky+=[
                round(sum(vars()[A][0][-x:]),1)
            ]
            
        # # # # # # # #  # # #
        HxPTS=df.iloc[i]['HxPTS']
        AxPTS=df.iloc[i]['AxPTS']
        FTR=df.iloc[i]['FTR']
        if HxPTS<=AxPTS:
            vars()[H][0]+=[0]
            one=AxPTS-HxPTS
            coef=one*(0 if FTR=='A' else 2 if FTR=='D' else 3) #pontos perdidos, neste caso pela AwayTeam
            vars()[A][0]+=[coef]
        else:
            vars()[A][0]+=[0]
            one=HxPTS-AxPTS
            coef=one*(0 if FTR=='H' else 2 if FTR=='D' else 3) #pontos perdidos, neste caso pela HomeTeam
            vars()[H][0]+=[coef]
            
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    # relação de diferença H-A
    unlucky_diff = list(map(lambda x,y:x-y,H_unlucky,A_unlucky))
    
    if diff:
        output=pd.DataFrame({'Unlucky_Diff'+str(x):unlucky_diff})
    else:
        output=pd.DataFrame({'Unlucky_H'+str(x):H_unlucky,
                            'Unlucky_A'+str(x):A_unlucky,
                            'Unlucky_Diff'+str(x):unlucky_diff})
        
    return output
            
        

In [336]:
def lucky(df,clubs,x,diff=True):
    for club in clubs:
        vars()[club]=[[],0] 
    H_lucky=[]
    A_lucky=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_lucky+=[0]
        else:
            H_lucky+=[
                round(sum(vars()[H][0][-x:]),1)
            ]
            
        if vars()[A][-1]<x:
            A_lucky+=[0]
        else:
            A_lucky+=[
                round(sum(vars()[A][0][-x:]),1)
            ]
            
        # #  # # # # # # # # # # # #
        HxPTS=df.iloc[i]['HxPTS']
        AxPTS=df.iloc[i]['AxPTS']
        FTR=df.iloc[i]['FTR']
        Hpts= 0 if FTR=='A' else 1 if FTR=='D' else 3
        Apts= 0 if FTR=='H' else 1 if FTR=='D' else 3
        coefH= Hpts-HxPTS
        coefA= Apts-AxPTS
        #vars()[H][0]+=[coefH if coefH>0 else 0]
        #vars()[A][0]+=[coefA if coefA>0 else 0]
        vars()[H][0]+=[coefH]
        vars()[A][0]+=[coefA]
        
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    #### lucky diff H-A
    lucky_diff=list(map(lambda x,y:x-y,H_lucky,A_lucky))
    
    if diff:
        output=pd.DataFrame({'Lucky_Diff'+str(x):lucky_diff})
    else:
        output=pd.DataFrame({'Lucky_H'+str(x):H_lucky,
                            'Lucky_A'+str(x):A_lucky,
                            'Lucky_Diff'+str(x):lucky_diff})
        
    return output        

In [337]:
def shot_quality(df,clubs,x,diff=True):
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 xG// 1 ST // 2 xG/ST
    H_quality=[]
    A_quality=[]
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        
        if vars()[H][-1]<x:
            H_quality+=[0]
        else:
            H_quality+=[round((sum(vars()[H][0][-x:])/(sum(vars()[H][1][-x:]))),1)]
        if vars()[A][-1]<x:
            A_quality+=[0]
        else:
            A_quality+=[round((sum(vars()[A][0][-x:])/(sum(vars()[A][1][-x:]))),1)]
            
        # # #  # # # # #  # # # #
        
        vars()[H][0]+=[df.iloc[i]['HxG']]
        vars()[H][1]+=[df.iloc[i]['HST']]
        
        
        vars()[A][0]+=[df.iloc[i]['AxG']]
        vars()[A][1]+=[df.iloc[i]['AST']]
        
        
        vars()[H][-1]+=1
        vars()[A][-1]+=1
        
    differ=list(map(lambda x,y:x-y,H_quality,A_quality))
    
    if diff:
        output=pd.DataFrame({'SQuality_Diff'+str(x):differ})
    else:
        output=pd.DataFrame({
            'SQuality_H'+str(x):H_quality,'SQuality_A'+str(x):A_quality,'SQuality_Diff'+str(x):differ
        })
        
    return output
        

In [338]:
def xpoints_made(df,clubs,x,diff=True):
    for club in clubs:
        vars()[club]=[[],[],[],0] #0 total de pts // 1 ppontos casa // 2 pontos fora
    H_xpoints=[[],[]] #0 total de pts // 1 pontos em casa
    A_xpoints=[[],[]]  #0 total de pontos // 1 pontos fora
    xpoints_diff=[[],[]] #0 diff total // 1 diff H-A
    #df=pd.read_excel(file+'.xlsx',0)
    size=len(df.index)
    for i in range(size):
        res=df.iloc[i]['FTR']
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
        HW=False
        D=False
        AW=False
        if vars()[H][-1]<x:
            H_xpoints[0]+=[0]
            H_xpoints[1]+=[0]
        else:
            H_xpoints[0]+=[round((sum(vars()[H][0])/len(vars()[H][0])),1)]
            H_xpoints[1]+=[round((sum(vars()[H][1])/len(vars()[H][1])),1)]
        if vars()[A][-1]<x:
            A_xpoints[0]+=[0]
            A_xpoints[1]+=[0]
        else:
            A_xpoints[0]+=[round((sum(vars()[A][0])/len(vars()[A][0])),1)]
            A_xpoints[1]+=[round((sum(vars()[A][2])/len(vars()[A][2])),1)]
            
        # trata da diff    
        #if vars()[H][-1]>=x and vars()[A][-1]>=x:
        #    xpoints_diff[0]+=[(sum(vars()[H][0])/len(vars()[H][0]))-(sum(vars()[A][0])/len(vars()[A][0]))] 
        #    xpoints_diff[1]+=[(sum(vars()[H][1])/len(vars()[H][1]))-(sum(vars()[A][2])/len(vars()[A][2]))]
        #else:
        #    xpoints_diff[0]+=[0]
        #    xpoints_diff[1]+=[0]
            
        # # # # #  # # # # #  # # #
        vars()[H][0]+=[df.iloc[i]['HxPTS']]
        vars()[H][1]+=[df.iloc[i]['HxPTS']]
        vars()[A][0]+=[df.iloc[i]['AxPTS']]
        vars()[A][2]+=[df.iloc[i]['AxPTS']]
        
        vars()[H][-1]+=1
        vars()[A][-1]+=1    
        
    xpoints_diff[0]=list(map(lambda x,y:x-y,H_xpoints[0],A_xpoints[0]))
    xpoints_diff[1]=list(map(lambda x,y:x-y,H_xpoints[1],A_xpoints[1]))
    
    if diff:
        output=pd.DataFrame({'xPoints_Diff_Total':xpoints_diff[0],'xPoints_Diff_H-A':xpoints_diff[1]})
    else:
        output=pd.DataFrame({'H_xPts_Total':H_xpoints[0],'H_xPts_H':H_xpoints[1],
                             'A_xPts_Total':A_xpoints[0],'A_xPts_A':A_xpoints[1],
                             'xPoints_Diff_Total':xpoints_diff[0],'xPoints_Diff_H-A':xpoints_diff[1],
                             })
    return output

In [339]:
def s_st_g_c(df,clubs,x,diff=True):
    for club in clubs:
        vars()[club]=[[],[],[],[],0]
    L_list=[[],[],[],[]] #0s,1st,2g,3c
    H_list=[[],[],[],[]]
    diff_list=[[],[],[],[]]
    l_rel=[[],[],[],[]] #relação >< que a média Low (1, maior q a media/0, menor/igual à média )
    h_rel=[[],[],[],[]] #relação >< que a média High
    #df=pd.read_excel(file+'.xlsx',0)
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
            
        
        if vars()[H][-1]<x:
            L_list[0]+=[0]
            L_list[1]+=[0]
            L_list[2]+=[0]
            L_list[3]+=[0]
            l_rel[0]+=[0]
            l_rel[1]+=[0]
            l_rel[2]+=[0]
            l_rel[3]+=[0]
        else:
            L_list[0]+=[int(round(sum(vars()[H][0][-x:])/x))]
            if (sum(vars()[H][0][-x:])/x) < vars()[H][0][-1]:
                l_rel[0]+=[1]
            else:
                l_rel[0]+=[0]
                
            L_list[1]+=[int(round(sum(vars()[H][1][-x:])/x))]
            if (sum(vars()[H][1][-x:])/x) < vars()[H][1][-1]:
                l_rel[1]+=[1]
            else:
                l_rel[1]+=[0]
                
            L_list[2]+=[int(round(sum(vars()[H][2][-x:])/x))]
            if (sum(vars()[H][2][-x:])/x) < vars()[H][2][-1]:
                l_rel[2]+=[1]
            else:
                l_rel[2]+=[0]
                
            L_list[3]+=[int(round(sum(vars()[H][3][-x:])/x))]
            if (sum(vars()[H][3][-x:])/x) < vars()[H][3][-1]:
                l_rel[3]+=[1]
            else:
                l_rel[3]+=[0]
            
        if vars()[A][-1]<x:
            H_list[0]+=[0]
            H_list[1]+=[0]
            H_list[2]+=[0]
            H_list[3]+=[0]
            h_rel[0]+=[0]
            h_rel[1]+=[0]
            h_rel[2]+=[0]
            h_rel[3]+=[0]
        else:
            H_list[0]+=[int(round(sum(vars()[A][0][-x:]))/x)]
            if (sum(vars()[A][0][-x:])/x) < vars()[A][0][-1]:
                h_rel[0]+=[1]
            else:
                h_rel[0]+=[0]
                
            H_list[1]+=[int(round(sum(vars()[A][1][-x:]))/x)]
            if (sum(vars()[A][1][-x:])/x) < vars()[A][1][-1]:
                h_rel[1]+=[1]
            else:
                h_rel[1]+=[0]
                
            H_list[2]+=[int(round(sum(vars()[A][2][-x:])/x))]
            if (sum(vars()[A][2][-x:])/x) < vars()[A][2][-1]:
                h_rel[2]+=[1]
            else:
                h_rel[2]+=[0]
                
            H_list[3]+=[int(round(sum(vars()[A][3][-x:])/x))]
            if (sum(vars()[A][3][-x:])/x) < vars()[A][3][-1]:
                h_rel[3]+=[1]
            else:
                h_rel[3]+=[0]
                
        # trata da relação de diferença     #   #   #  #  #   #   #  #   #  #  #   #   
        if vars()[H][-1]>=x and vars()[A][-1]>=x:
            diff_list[0]+=[int(round((sum(vars()[H][0][-x:])/x)-(sum(vars()[A][0][-x:])/x)))]
            diff_list[1]+=[int(round((sum(vars()[H][1][-x:])/x)-(sum(vars()[A][1][-x:])/x)))]
            diff_list[2]+=[int(round((sum(vars()[H][2][-x:])/x)-(sum(vars()[A][2][-x:])/x)))]
            diff_list[3]+=[int(round((sum(vars()[H][3][-x:])/x)-(sum(vars()[A][3][-x:])/x)))]
        else:
            diff_list[0]+=[0]
            diff_list[1]+=[0]
            diff_list[2]+=[0]
            diff_list[3]+=[0]
        # # ###   ##     #     #   #  #  #   #   #  #   #  #   #  #  #  #   #  #  #         
        vars()[H][0]+=[df.iloc[i]['HS']]
        vars()[A][0]+=[df.iloc[i]['AS']]
        vars()[H][1]+=[df.iloc[i]['HST']]
        vars()[A][1]+=[df.iloc[i]['AST']]
        vars()[H][2]+=[df.iloc[i]['FTHG']]
        vars()[A][2]+=[df.iloc[i]['FTAG']]
        vars()[H][3]+=[df.iloc[i]['FTAG']]
        vars()[A][3]+=[df.iloc[i]['FTHG']]
        
        
        vars()[H][-1]+=1
        vars()[A][-1]+=1
            
            
    if diff:
        output=pd.DataFrame({'S_Diff_H-A_'+str(x):diff_list[0],'ST_Diff_H-A_'+str(x):diff_list[1],'Scrd_Diff_H-A_'+str(x):diff_list[2],'Concd_Diff_H-A_'+str(x):diff_list[3]})
    else:
        output=pd.DataFrame({'HS_'+str(x):L_list[0],'Last_H_S':l_rel[0],'HST_'+str(x):L_list[1],'Last_H_ST':l_rel[1],
                             'HScrd_'+str(x):L_list[2],'Last_H_Scrd':l_rel[2],'HConcd_'+str(x):L_list[3],'Last_H_Concd':l_rel[3],
                            'AS_'+str(x):H_list[0],'Last_A_S':h_rel[0],'AST_'+str(x):H_list[1],'Last_A_ST':h_rel[1],
                             'AScrd_'+str(x):H_list[2],'Last_A_Scrd':h_rel[2],'AConcd_'+str(x):H_list[3],'Last_A_Concd':h_rel[3],
                            'S_Diff_H-A_'+str(x):diff_list[0],'ST_Diff_H-A_'+str(x):diff_list[1],
                             'Scrd_Diff_H-A_'+str(x):diff_list[2],'Concd_Diff_H-A_'+str(x):diff_list[3]})
            
    return output 

In [340]:
def non_odd_counter(df,clubs):
    for club in clubs:
        vars()[club]=[0,0,0,0,0,0,0,0,0,0] #-1 counter ||| 0 Win, 1 Scrd, 2 Concd,3/4 WinH/A,5/6 ScrdH/A,7/8ConcdH/A
    L_count=[[],[],[]]
    H_count=[[],[],[]]
    HA_count=[[],[],[],[],[],[]]
    Draw_count=[[],[],[],[]] #AINDA NÃOOOOOOOOOOOOOOOOOOO ADICIONEIIIIIIIIIIIIIIIIIII
    #df=pd.read_excel(file+'.xlsx',0)    
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
               
        L_count[0]+=[vars()[H][0]]
        L_count[1]+=[vars()[H][1]]
        L_count[2]+=[vars()[H][2]]
        
        H_count[0]+=[vars()[A][0]]
        H_count[1]+=[vars()[A][1]]
        H_count[2]+=[vars()[A][2]]
        
        HA_count[0]+=[vars()[H][3]]
        HA_count[1]+=[vars()[A][4]]
        HA_count[2]+=[vars()[H][5]]
        HA_count[3]+=[vars()[A][6]]
        HA_count[4]+=[vars()[H][7]]
        HA_count[5]+=[vars()[A][8]]
        
        
        #if df.iloc[i]['Lowest_Odd']=='H':   #Odd + baixa CASA 
        if df.iloc[i]['FTR']=='H': #se a equipa de casa ganha, como fica o counter
            if vars()[H][0]>0:
                vars()[H][0]+=1 
            else:
                vars()[H][0]=1
            if vars()[A][0]>0:
                vars()[A][0]=-1 
            else:
                vars()[A][0]-=1 
            #H/A-> focando só nas prestações H/A
            if vars()[H][3]>0:
                vars()[H][3]+=1 
            else:
                vars()[H][3]=1
            if vars()[A][4]>0:
                vars()[A][4]=-1 
            else:
                vars()[A][4]-=1
        if df.iloc[i]['FTR']=='A': #se a equipa de fora ganha, como fica o counter
            if vars()[H][0]>0:
                vars()[H][0]=-1 
            else:
                vars()[H][0]-=1
            if vars()[A][0]>0:
                vars()[A][0]+=1 
            else:
                vars()[A][0]=1 
            #H/A-> focando só nas prestações H/A    
            if vars()[H][3]>0:
                vars()[H][3]=-1 
            else:
                vars()[H][3]-=1
            if vars()[A][4]>0:
                vars()[A][4]+=1 
            else:
                vars()[A][4]=1 
        if df.iloc[i]['FTR']=='D': #se há empate, os counter de jogos ganhos vão todos a zero
            if vars()[H][0]>0:
                vars()[H][0]=-1 
            else:
                vars()[H][0]-=1
            if vars()[A][0]>0:
                vars()[A][0]=-1 
            else:
                vars()[A][0]-=1 
            #H/A-> focando só nas prestações H/A       
            if vars()[H][3]>0:
                vars()[H][3]=-1 
            else:
                vars()[H][3]-=1
            if vars()[A][4]>0:
                vars()[A][4]=-1 
            else:
                vars()[A][4]-=1 
        if df.iloc[i]['FTHG']>0 and df.iloc[i]['FTAG']>0: # se ambas marcam
            if vars()[H][1]>0:
                vars()[H][1]+=1 
            else:
                vars()[H][1]=1    
            if vars()[A][1]>0:
                vars()[A][1]+=1    
            else:
                vars()[A][1]=1    
            if vars()[H][2]>0:
                vars()[H][2]=-1
            else:
                vars()[H][2]-=1
            if vars()[A][2]>0:
                vars()[A][2]=-1 
            else:
                vars()[A][2]-=1 
            #H/A-> focando só nas prestações H/A   
            if vars()[H][5]>0:
                vars()[H][5]+=1 
            else:
                vars()[H][5]=1    
            if vars()[A][6]>0:
                vars()[A][6]+=1    
            else:
                vars()[A][6]=1    
            if vars()[H][7]>0:
                vars()[H][7]=-1
            else:
                vars()[H][7]-=1
            if vars()[A][8]>0:
                vars()[A][8]=-1 
            else:
                vars()[A][8]-=1 
        if df.iloc[i]['FTHG']>0 and df.iloc[i]['FTAG']==0: # se só casa marca
            #print('(+,0)')
            if vars()[H][1]>0:
                vars()[H][1]+=1 
            else:
                vars()[H][1]=1    
            if vars()[A][1]>0:
                vars()[A][1]=-1    
            else:
                vars()[A][1]-=1    
            if vars()[H][2]>0:
                vars()[H][2]+=1
            else:
                vars()[H][2]=1
            if vars()[A][2]>0:
                vars()[A][2]=-1 
            else:
                vars()[A][2]-=1 
            #H/A-> focando só nas prestações H/A
            if vars()[H][5]>0:
                vars()[H][5]+=1 
            else:
                vars()[H][5]=1    
            if vars()[A][6]>0:
                vars()[A][6]=-1    
            else:
                vars()[A][6]-=1    
            if vars()[H][7]>0:
                vars()[H][7]+=1
            else:
                vars()[H][7]=1
            if vars()[A][8]>0:
                vars()[A][8]=-1 
            else:
                vars()[A][8]-=1 
        if df.iloc[i]['FTHG']==0 and df.iloc[i]['FTAG']>0: # se só fora marca
            #print('(0,+)')
            if vars()[H][1]>0:
                vars()[H][1]=-1 
            else:
                vars()[H][1]-=1    
            if vars()[A][1]>0:
                vars()[A][1]+=1    
            else:
                vars()[A][1]=1    
            if vars()[H][2]>0:
                vars()[H][2]=-1
            else:
                vars()[H][2]-=1
            if vars()[A][2]>0:
                vars()[A][2]+=1 
            else:
                vars()[A][2]=1 
            #H/A-> focando só nas prestações H/A
            if vars()[H][5]>0:
                vars()[H][5]=-1 
            else:
                vars()[H][5]-=1    
            if vars()[A][6]>0:
                vars()[A][6]+=1    
            else:
                vars()[A][6]=1    
            if vars()[H][7]>0:
                vars()[H][7]=-1
            else:
                vars()[H][7]-=1
            if vars()[A][8]>0:
                vars()[A][8]+=1 
            else:
                vars()[A][8]=1
        if df.iloc[i]['FTHG']==0 and df.iloc[i]['FTAG']==0: # se nenhuma marca
            #print('(0,0)')
            if vars()[H][1]>0:
                vars()[H][1]=-1 
            else:
                vars()[H][1]-=1    
            if vars()[A][1]>0:
                vars()[A][1]=-1    
            else:
                vars()[A][1]-=1    
            if vars()[H][2]>0:
                vars()[H][2]+=1
            else:
                vars()[H][2]=1
            if vars()[A][2]>0:
                vars()[A][2]+=1 
            else:
                vars()[A][2]=1 
            #H/A-> focando só nas prestações H/A
            if vars()[H][5]>0:
                vars()[H][5]=-1 
            else:
                vars()[H][5]-=1    
            if vars()[A][6]>0:
                vars()[A][6]=-1    
            else:
                vars()[A][6]-=1    
            if vars()[H][7]>0:
                vars()[H][7]+=1
            else:
                vars()[H][7]=1
            if vars()[A][8]>0:
                vars()[A][8]+=1 
            else:
                vars()[A][8]=1
        
        
    #Total, regista todos os jogos
    #_H/A regista o registo em casa de H e fora de A
    output=pd.DataFrame({'WinCounterTotal_H':L_count[0],'WinCounterTotal_A':H_count[0],
                         'WinCounter_H':HA_count[0],'WinCounter_A':HA_count[1],
                         'ScrdCounterTotal_H':L_count[1],'ScrdCounterTotal_A':H_count[1],
                         'ScrdCounter_H':HA_count[2],'ScrdCounter_A':HA_count[3],
                         'ConcdCounterTotal_H':L_count[2],'ConcdCounterTotal_A':H_count[2],
                         'ConcdCounter_H':HA_count[4],'ConcdCounter_A':HA_count[5]
                        })
    return output

In [341]:
def lowest_lost(df,clubs):
    for club in clubs:
        vars()[club]=1000
    llost=[]
    size=len(df.index)
    for i in range(size):
        H=False
        A=False
        if df.iloc[i]['Lowest_Odd']=='H':
            L=df.iloc[i]['HomeTeam']
            Hi=df.iloc[i]['AwayTeam'] #high odd
            H=True
        else:
            L=df.iloc[i]['AwayTeam']
            Hi=df.iloc[i]['HomeTeam'] #high odd
            A=True
            
        llost+=[round_down(vars()[L],1)] #uso o round_down(,1)
        
        if H and (df.iloc[i]['FTR']!='H'):
            if df.iloc[i]['ODDH_Aver.']<vars()[L]:
                vars()[L]=df.iloc[i]['ODDH_Aver.']
        elif A and (df.iloc[i]['FTR']!='A'):
            if df.iloc[i]['ODDA_Aver.']<vars()[L]:
                vars()[L]=df.iloc[i]['ODDA_Aver.']
                
        if H and (df.iloc[i]['FTR']!='A'):
            if df.iloc[i]['ODDA_Aver.']<vars()[Hi]:
                vars()[Hi]=df.iloc[i]['ODDA_Aver.']
        elif A and (df.iloc[i]['FTR']!='H'):
            if df.iloc[i]['ODDH_Aver.']<vars()[Hi]:
                vars()[Hi]=df.iloc[i]['ODDH_Aver.']
                
    output=pd.DataFrame({'L_LowestLost':llost})
    return output

In [342]:
def won_notwon(df):
    size=len(df.index)
    wnw=[]
    for i in range(size):
        if df.iloc[i]['FTR']=='H':
            wnw+=[1]
        else:
            wnw+=[0]
                
    output=pd.DataFrame({'Won/NotWon':wnw})
    return output

In [343]:
def x_pass(df,clubs,x,diff=True):
    #média dos últimos x jogos de:
    #xG, xPTS, Pass%, PPDA, DEEP, CCC, xGConcd
    for club in clubs:
        vars()[club]=[[],[],[],[],[],[],[],0]
    L_list=[[],[],[],[],[],[],[]] #0xG,1xPTS,2Pass%,3PPDA,4DEEP,5CCC
    H_list=[[],[],[],[],[],[],[]]
    diff_list=[[],[],[],[],[],[],[]]
    l_rel=[[],[],[],[],[],[],[]] #relação >< que a média Low (1, maior q a media/0, menor/igual à média )
    h_rel=[[],[],[],[],[],[],[]] #relação >< que a média High
    #df=pd.read_excel(file+'.xlsx',0)
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
            
        
        if vars()[H][-1]<x:
            L_list[0]+=[0]
            L_list[1]+=[0]
            L_list[2]+=[0]
            L_list[3]+=[0]
            L_list[4]+=[0]
            L_list[5]+=[0]
            L_list[6]+=[0]
            l_rel[0]+=[0]
            l_rel[1]+=[0]
            l_rel[2]+=[0]
            l_rel[3]+=[0]
            l_rel[4]+=[0]
            l_rel[5]+=[0]
            l_rel[6]+=[0]
        else:
            L_list[0]+=[int(round(sum(vars()[H][0][-x:])/x))]
            if (sum(vars()[H][0][-x:])/x) < vars()[H][0][-1]:
                l_rel[0]+=[1]
            else:
                l_rel[0]+=[0]
                
            L_list[1]+=[round(sum(vars()[H][1][-x:])/x,1)]
            if (sum(vars()[H][1][-x:])/x) < vars()[H][1][-1]:
                l_rel[1]+=[1]
            else:
                l_rel[1]+=[0]
                
            L_list[2]+=[int(round(sum(vars()[H][2][-x:])/x))]
            if (sum(vars()[H][2][-x:])/x) < vars()[H][2][-1]:
                l_rel[2]+=[1]
            else:
                l_rel[2]+=[0]
                
            L_list[3]+=[int(round(sum(vars()[H][3][-x:])/x))]
            if (sum(vars()[H][3][-x:])/x) < vars()[H][3][-1]:
                l_rel[3]+=[1]
            else:
                l_rel[3]+=[0]
            
            L_list[4]+=[int(round(sum(vars()[H][4][-x:])/x))]
            if (sum(vars()[H][4][-x:])/x) < vars()[H][4][-1]:
                l_rel[4]+=[1]
            else:
                l_rel[4]+=[0]
            
            L_list[5]+=[int(round(sum(vars()[H][5][-x:])/x))]
            if (sum(vars()[H][5][-x:])/x) < vars()[H][5][-1]:
                l_rel[5]+=[1]
            else:
                l_rel[5]+=[0]
                
            L_list[6]+=[int(round(sum(vars()[H][6][-x:])/x))]
            if (sum(vars()[H][6][-x:])/x) < vars()[H][6][-1]:
                l_rel[6]+=[1]
            else:
                l_rel[6]+=[0]
            
        if vars()[A][-1]<x:
            H_list[0]+=[0]
            H_list[1]+=[0]
            H_list[2]+=[0]
            H_list[3]+=[0]
            H_list[4]+=[0]
            H_list[5]+=[0]
            H_list[6]+=[0]
            h_rel[0]+=[0]
            h_rel[1]+=[0]
            h_rel[2]+=[0]
            h_rel[3]+=[0]
            h_rel[4]+=[0]
            h_rel[5]+=[0]
            h_rel[6]+=[0]
        else:
            H_list[0]+=[int(round(sum(vars()[A][0][-x:])/x))]
            if (sum(vars()[A][0][-x:])/x) < vars()[A][0][-1]:
                h_rel[0]+=[1]
            else:
                h_rel[0]+=[0]
                
            H_list[1]+=[round(sum(vars()[A][1][-x:])/x,1)]
            if (sum(vars()[A][1][-x:])/x) < vars()[A][1][-1]:
                h_rel[1]+=[1]
            else:
                h_rel[1]+=[0]
            H_list[2]+=[int(round(sum(vars()[A][2][-x:])/x))]
            if (sum(vars()[A][2][-x:])/x) < vars()[A][2][-1]:
                h_rel[2]+=[1]
            else:
                h_rel[2]+=[0]
                
            H_list[3]+=[int(round(sum(vars()[A][3][-x:])/x))]
            if (sum(vars()[A][3][-x:])/x) < vars()[A][3][-1]:
                h_rel[3]+=[1]
            else:
                h_rel[3]+=[0]
                
            H_list[4]+=[int(round(sum(vars()[A][4][-x:])/x))]
            if (sum(vars()[A][4][-x:])/x) < vars()[A][4][-1]:
                h_rel[4]+=[1]
            else:
                h_rel[4]+=[0]
                
            H_list[5]+=[int(round(sum(vars()[A][5][-x:])/x))]
            if (sum(vars()[A][5][-x:])/x) < vars()[A][5][-1]:
                h_rel[5]+=[1]
            else:
                h_rel[5]+=[0]
                
            H_list[6]+=[int(round(sum(vars()[A][6][-x:])/x))]
            if (sum(vars()[A][6][-x:])/x) < vars()[A][6][-1]:
                h_rel[6]+=[1]
            else:
                h_rel[6]+=[0]    
        # trata da relação de diferença     #   #   #  #  #   #   #  #   #  #  #   #   
        if vars()[H][-1]>=x and vars()[A][-1]>=x:
            diff_list[0]+=[int(round((sum(vars()[H][0][-x:])/x)-(sum(vars()[A][0][-x:])/x)))]
            diff_list[1]+=[round((sum(vars()[H][1][-x:])/x)-(sum(vars()[A][1][-x:])/x),1)]
            diff_list[2]+=[int(round((sum(vars()[H][2][-x:])/x)-(sum(vars()[A][2][-x:])/x)))]
            diff_list[3]+=[int(round((sum(vars()[H][3][-x:])/x)-(sum(vars()[A][3][-x:])/x)))]
            diff_list[4]+=[int(round((sum(vars()[H][4][-x:])/x)-(sum(vars()[A][4][-x:])/x)))]
            diff_list[5]+=[int(round((sum(vars()[H][5][-x:])/x)-(sum(vars()[A][5][-x:])/x)))]
            diff_list[6]+=[int(round((sum(vars()[H][6][-x:])/x)-(sum(vars()[A][6][-x:])/x)))]
        else:
            diff_list[0]+=[0]
            diff_list[1]+=[0]
            diff_list[2]+=[0]
            diff_list[3]+=[0]
            diff_list[4]+=[0]
            diff_list[5]+=[0]
            diff_list[6]+=[0]
        # # ###   ##     #     #   #  #  #   #   #  #   #  #   #  #  #  #   #  #  #     
            
        
        vars()[H][0]+=[df.iloc[i]['HxG']]
        vars()[A][0]+=[df.iloc[i]['AxG']]
        vars()[H][1]+=[df.iloc[i]['HxPTS']]
        vars()[A][1]+=[df.iloc[i]['AxPTS']]
        vars()[H][2]+=[df.iloc[i]['HPass']]
        vars()[A][2]+=[df.iloc[i]['APass']]
        vars()[H][3]+=[df.iloc[i]['HPPDA']]
        vars()[A][3]+=[df.iloc[i]['APPDA']]
        vars()[H][4]+=[df.iloc[i]['HDEEP']]
        vars()[A][4]+=[df.iloc[i]['ADEEP']]
        vars()[H][5]+=[df.iloc[i]['HCCC']]
        vars()[A][5]+=[df.iloc[i]['ACCC']]
        vars()[H][6]+=[df.iloc[i]['AxG']]
        vars()[A][6]+=[df.iloc[i]['HxG']]
            
               
        vars()[H][-1]+=1
        vars()[A][-1]+=1
            
    
    if diff:
        output=pd.DataFrame({'xG_Diff_'+str(x):diff_list[0],'xPTS_Diff_'+str(x):diff_list[1],
                             'Pass%_Diff_'+str(x):diff_list[2],'PPDA_Diff_'+str(x):diff_list[3],
                             'DEEP_Diff_'+str(x):diff_list[4],'CCC_Diff_'+str(x):diff_list[5],
                             'xConcd_Diff_'+str(x):diff_list[6]
                            })
    else:
        output=pd.DataFrame({'HxG_'+str(x):L_list[0],'Last_HxG':l_rel[0],'HxPTS_'+str(x):L_list[1],'Last_HxPTS':l_rel[1],
                            'HPass%_'+str(x):L_list[2],'Last_HPass%':l_rel[2],'HPPDA_'+str(x):L_list[3],'Last_HPPDA':l_rel[3],
                            'HDEEP_'+str(x):L_list[4],'Last_HDEEP':l_rel[4],
                            'HCCC_'+str(x):L_list[5],'Last_HCCC':l_rel[5],
                            'HxConcd_'+str(x):L_list[6],'Last_HxConcd':l_rel[6],
                            'AxG_'+str(x):H_list[0],'Last_AxG':h_rel[0],'AxPTS_'+str(x):H_list[1],'Last_AxPTS':h_rel[1],
                            'APass%_'+str(x):H_list[2],'Last_APass%':h_rel[2],'APPDA_'+str(x):H_list[3],'Last_APPDA':h_rel[3],
                            'ADEEP_'+str(x):H_list[4],'Last_ADEEP':h_rel[4],
                            'ACCC_'+str(x):H_list[5],'Last_ACCC':h_rel[5],
                            'AxConcd_'+str(x):H_list[6],'Last_AxConcd':h_rel[6],
                             ####
                            'xG_Diff_'+str(x):diff_list[0],'xPTS_Diff_'+str(x):diff_list[1],
                            'Pass%_Diff_'+str(x):diff_list[2],'PPDA_Diff_'+str(x):diff_list[3],
                            'DEEP_Diff_'+str(x):diff_list[4],'CCC_Diff_'+str(x):diff_list[5],
                            'xConcd_Diff_'+str(x):diff_list[6]
                            })
            
    return output 

In [344]:
def HA_points(df,clubs,x,diff=True):
    #média de pontos feitos em casa e fora
    #faz se a média de todos os jogos até x,
    #dps de x, faz-se a média dos últimos x jogos
    for club in clubs:
        vars()[club]=[[],[],0,0] #2Hcounter, 3Acounter
    HA_list=[[],[]] #0H,1A
    size=len(df.index)
    for i in range(size):
        H=df.iloc[i]['HT']
        A=df.iloc[i]['AT']
            
        if vars()[H][2]==0:
            HA_list[0]+=[0]
        elif vars()[H][2]<x:
            HA_list[0]+=[round((sum(vars()[H][0])/len(vars()[H][0])),1)]
        elif vars()[H][2]>=x:
            HA_list[0]+=[round((sum(vars()[H][0][(-x):])/x),1)]
        else:
            print('Erro')
            raise('HA_points')
            
        if vars()[A][3]==0:
            HA_list[1]+=[0]
        elif vars()[A][3]<x:
            HA_list[1]+=[round((sum(vars()[A][1])/len(vars()[A][1])),1)]
        elif vars()[A][3]>=x:
            HA_list[1]+=[round((sum(vars()[A][1][(-x):])/x),1)]
        else:
            print('Erro')
            raise('HA_points')
            
            
        #trata de acrescentar à lista
        if df.iloc[i]['FTR']=='H':
            vars()[H][0]+=[3]
            vars()[A][1]+=[0]
        elif df.iloc[i]['FTR']=='D':
            vars()[H][0]+=[1]
            vars()[A][1]+=[1]
        else:
            vars()[H][0]+=[0]
            vars()[A][1]+=[3]
            
        
        vars()[H][2]+=1
        vars()[A][3]+=1
        
    
    HAAH=list(map(lambda x,y:x-y,HA_list[0],HA_list[1]))
    output=pd.DataFrame({'HT_HPoints_'+str(x):HA_list[0],'AT_APoints_'+str(x):HA_list[1],'HA_PointsDiff':HAAH})
                
    return output       
    

In [345]:
def join_husky(df,clubs,x,diff=True,read=False):
    
    if read:
        df=pd.read_excel(df+'.xlsx',0)
        
    league=df['League']
    ht=df['HT']
    at=df['AT']
    date=df['Date']
    time=df['Time']
    try:
        oddh=df['ODDH_Aver.']
        oddd=df['ODDD_Aver.']
        odda=df['ODDA_Aver.']
    except:
        print('Não usou as Odds')
        pass
    print('done extra')
    a=xpoints_made(df,clubs,x,diff=diff)
    print('done a')
    b=HA_points(df,clubs,x,diff=diff)
    print('done b')
    c=x_pass(df,clubs,x,diff=diff)
    print('done c')
    d=shot_quality(df,clubs,x,diff=diff)
    print('done d')
    e=s_st_g_c(df,clubs,x,diff=diff)
    print('done e')
    f=non_odd_counter(df,clubs)
    print('done f')
    g=lucky(df,clubs,x,diff=diff)
    print('done g')
    h=unlucky(df,clubs,x,diff=diff)
    print('done h')
    i=ST_form(df,clubs,x,diff=diff)
    print('done i')
    j=CCC_form(df,clubs,x,diff=diff)
    print('done j')
    k=DEEP_form(df,clubs,x,diff=diff)
    print('done k')
    l=PPDA_form(df,clubs,x,diff=diff)
    print('done l')
    m=Pass_form(df,clubs,x,diff=diff)
    print('done m')
    n=Def_form(df,clubs,x,diff=diff)
    print('done n')
    o=Atack_form(df,clubs,x,diff=diff)
    print('done o')
    p=form1(df,clubs,x,diff=diff)
    print('done p')
    q=form2(df,clubs,x,diff=diff)
    print('done q')
    r=form3(df,clubs,x,diff=diff)
    print('done r')
    s=new_form(df,clubs,x,diff=diff)
    print('done s')
    t=won_notwon(df)
    print('done t')
    try:
        df1=pd.concat([league,date,time,ht,at,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,oddh,oddd,odda],sort=False,axis=1)
    except:
        df1=pd.concat([league,date,time,ht,at,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r],sort=False,axis=1)
    df1=df1.sort_values(['Date','Time'],ascending=[True,True])
    return df1

In [346]:
#clubs=findall_clubs2_new('NewPL14_15')

In [347]:
#dfPL=pd.read_excel('NewPL14_15.xlsx')

In [348]:
#test1=join_husky(dfPL,clubs,5,diff=False)

In [349]:
#test1.to_excel('HuskyTestPL14_15.xlsx',index=False)

In [350]:
def alpha_concat(files): #faz concat da lista de files e ordena-os já por date e time
    df1=pd.concat(files,sort=False)
    df=df1.sort_values(['Date'], ascending=True)
    return df

Este `new_cut_zeros` só serve para criar os ficheiros que juntam épocas de várias ligas  

In [351]:
def new_cut_zeros(file,x): #retira os dados com counters/médias/pontos a zero
    #precisamos de indicar o x dos jogos para a média
    size=len(file.index) #vamos só usar uma coluna, cuidado com erros
    i=0
    done=False
    conter=0
    ind=[]
    while i<size and not(done):
        if not((file.iloc[i]['Pass%_Diff_'+str(x)]==0) and (file.iloc[i]['S_Diff'+str(x)]==0) and (file.iloc[i]['ST_Diff'+str(x)]==0)):
            ind+=[i]
            counter+=1
        else:
            counter=0
        i+=1
        if counter==100: #se encontrarmos 100 jogos seguidos sem zeros, acaba o cicl0
            done=True
    conc=[]
    for n in ind:
        conc+=[pd.DataFrame(file.iloc[n,:]).T]
    conc+=[file[i:]]
    df=pd.concat(conc,sort=False)
    return df

`new_cut_zeros2` é para só para ser usado numa época de uma só liga

In [352]:
def new_cut_zeros2(file,x):
    #precisamos de indicar o x dos jogos para a média
    size=len(file.index) #vamos só usar uma coluna, cuidado com erros
    i=0
    done=False
    conter=0
    ind=[]
    while i<size and not(done):
        if not((file.iloc[i]['Pass%_Diff_'+str(x)]==0) and (file.iloc[i]['S_Diff_H-A_'+str(x)]==0) and (file.iloc[i]['ST_Diff_H-A_'+str(x)]==0)):
            ind+=[i]
            counter+=1
        else:
            counter=0
        i+=1
        if counter==25: #se encontrarmos 100 jogos seguidos sem zeros, acaba o cicl0
            done=True
    conc=[]
    for n in ind:
        conc+=[pd.DataFrame(file.iloc[n,:]).T]
    conc+=[file[i:]]
    df=pd.concat(conc,sort=False)
    return df

In [353]:
def husky_pos(df,clubs,x,diff=True,read=True):
    df=join_husky(df,clubs,x,diff=diff,read=read)
    df=new_cut_zeros2(df,x)
    return df

`year_husky` junta num só ficheiro tds as ligas de uma época. 

In [354]:
def year_husky(os_path,x,season,diff=True,read=True): 
    #junta num só ficheiro tds as ligas de uma época
    import os
    ind=new_find_data(os_path)
    rep=[] #repositorio para fazer concat
    for data_file in ind:
        try:
            season_list=os.listdir(os_path+data_file+'\\')
            i=season_index(os_path+data_file+'\\',season)
            #print(os_path+data_file+'\\'+season_list[i][:-5])
            clubs=findall_clubs2_new(os_path+data_file+'\\'+season_list[i][:-5])
            print(os_path+data_file+'\\'+season_list[i][:-5])
            file=husky_pos(os_path+data_file+'\\'+season_list[i][:-5],clubs,x,diff=diff,read=read)
            rep+=[file]
        except:
            print('except')
            print(data_file,'season:',season)
        #    pass
    print(len(rep))
    df=pd.concat(rep,sort=False)
    df=df.sort_values(['Date','Time'],ascending=[True,True])
    df=df.reset_index(drop=True)
    return df

`league_husky` junta num só ficheiro as épocas que queremos de uma liga

In [355]:
def league_husky(path,f_season,l_season,x,diff=True,read=True):
    os_path='C:\\Users\\joaom\\Documents\\Projetos\\PYTHON\\Apostas\\'+path+'___\\'
    i1=season_index(os_path,f_season)
    i2=season_index(os_path,l_season)
    seasons=os.listdir(os_path)
    rep=[]
    for i in range(i1,i2+1):
        clubs=findall_clubs2_new(os_path+seasons[i][:-5])
        file=husky_pos(os_path+seasons[i][:-5],clubs,x,diff=diff,read=read)
        rep+=[file]
    df=pd.concat(rep,sort=False)
    df=df.sort_values(['Date','Time'],ascending=[True,True])
    df=df.reset_index(drop=True)
    return df
    

`year_complete` junta o AllOddCounter, e faz o cut_zeros (e também põe o Won/NotWon na última coluna).

Recebe o file de `year_alpha` 

Como não temos Time, os jogos não estão bem por ordem (entre ligas), por iss, nesses casos não vamos usar all_odd_counter

In [356]:
def year_complete(file,x,odd_allcounter=False):
    if odd_allcounter:
        file=odd_allcounter_alpha(file)
        colunas=list(file.columns.values)
        file=file[colunas[:-2]+[colunas[-1]]+[colunas[-2]]]
        file=cut_zeros(file,x)
        file=file.reset_index(drop=True)
    else:
        file=cut_zeros(file,x)
        file=file.reset_index(drop=True)
    return file

`all_alpha` prepara o ficheiro para passar para as pastas do PC e faz o `year_alpha` para os anos que queremos

In [357]:
def all_husky(os_path,x,f_season,l_season,diff=True,read=True):
    rep=[]
    for i in range(f_season,l_season+1):
        df=year_husky(os_path,x,i,diff=diff,read=read)
        rep+=[df]
        print('---'+str(i)+'----')
    bigfile=pd.concat(rep,sort=False)
    bigfile=bigfile.sort_values(['Date','Time'],ascending=[True,True])
    bigfile=bigfile.reset_index(drop=True)
    return bigfile

____________________

### Let the tests begin

In [41]:
bund15=pd.read_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//NewBundesliga___//NewBund14_15.xlsx',index=False)

In [42]:
bund15

,League,Date,Time,Attendance,HT,AT,FTR,FTHG,FTAG,HxG,AxG,HxPTS,AxPTS,HP,AP,HS,AS,HST,AST,HSOT,ASOT,HBS,ABS,HCCC,ACCC,HPass,APass,HDEEP,ADEEP,HPPDA,APPDA,HTack,ATack,HAD,AAD,HCorn,ACorn,HOff,AOff,HSaves,ASaves,HF,AF,HY,AY,HR,AR
0,German Bundesliga,2014-08-22,1930,"71,000.",Bayern Munich,Wolfsburg,H,2,1,2.57,1.20,2.35,0.47,59.8,40.2,17,9,10,4,3,4,4,1,5,2,85.0,77.3,5,4,9.63,21.85,84.2,87.5,57.4,42.6,9,3,3,3,3,8,5,5,0,2,0,0
1,German Bundesliga,2014-08-23,1430,"23,792.",Hoffenheim,FC Augsburg,H,2,0,1.53,0.28,2.51,0.31,42.8,57.2,11,12,3,2,5,8,3,2,1,0,61.8,69.9,6,2,6.59,5.74,85.7,65.6,50.9,49.1,7,3,1,0,2,1,19,19,2,3,0,0
2,German Bundesliga,2014-08-23,1430,"59,672.",Hertha Berlin,Werder Bremen,D,2,2,1.76,1.19,1.83,0.85,54.2,45.8,19,9,5,5,7,3,7,1,1,1,79.8,77.1,3,5,5.69,9.87,77.3,83.3,49.0,51.0,9,5,4,6,3,3,17,17,1,2,0,0
3,German Bundesliga,2014-08-23,1430,"46,200.",Eintracht Frankfurt,SC Freiburg,H,1,0,1.75,1.38,1.65,1.10,36.1,63.9,10,13,3,6,5,1,2,6,1,2,70.5,82.2,3,13,10.52,3.47,81.1,80.8,53.7,46.3,6,5,2,1,6,2,17,17,1,3,0,0
4,German Bundesliga,2014-08-23,1430,"50,000.",Cologne,Hamburg,D,0,0,0.52,0.91,0.94,1.70,45.9,54.1,10,19,3,4,4,11,3,4,1,1,71.8,77.4,4,2,13.08,7.07,71.4,73.3,47.9,52.1,3,6,2,2,4,3,23,23,2,2,0,0
5,German Bundesliga,2014-08-23,1430,"49,000.",Hannover 96,Schalke,H,2,1,1.18,0.96,1.57,1.12,44.1,55.9,8,12,3,5,2,4,3,3,2,1,68.9,72.4,4,3,9.64,6.06,78.6,77.4,60.0,40.0,2,4,1,2,4,1,12,12,2,1,0,0
6,German Bundesliga,2014-08-23,1730,"80,667.",Borussia Dortmund,Bayer Leverkusen,A,0,2,0.48,1.15,0.74,1.95,58.5,41.5,11,11,4,2,3,5,4,4,0,2,70.9,59.2,2,5,3.33,5.80,78.9,65.2,47.4,52.6,8,4,1,5,0,4,13,13,1,2,0,0
7,German Bundesliga,2014-08-24,1630,"50,203.",M'gladbach,Stuttgart,D,1,1,2.41,0.43,2.69,0.20,56.0,44.0,20,7,6,3,12,1,2,3,4,0,87.0,80.8,7,3,14.11,14.50,72.2,84.6,59.2,40.8,6,3,3,2,2,5,8,8,2,1,0,0
8,German Bundesliga,2014-08-29,1930,"30,660.",FC Augsburg,Borussia Dortmund,A,2,3,1.00,2.30,0.55,2.26,53.7,46.3,12,22,3,9,7,6,2,7,2,3,82.4,76.5,2,8,9.04,13.81,75.0,73.7,50.0,50.0,3,6,8,2,6,1,9,9,2,3,0,0
9,German Bundesliga,2014-08-30,1430,"55,000.",Stuttgart,Cologne,A,0,2,0.72,1.30,0.82,1.88,68.3,31.7,14,11,5,5,5,3,4,3,1,1,86.7,70.1,9,4,6.30,26.00,93.3,92.9,60.7,39.3,7,4,5,1,3,5,11,11,1,2,0,0


In [370]:
Husky_LL2=league_husky('NewBundesliga',2015,2020,5,diff=False)

done extra
done a
done b
done c
done d
done e
done f
done g
done h
done i
done j
done k
done l
done m
Mainz HomeTeam, Def_form, I
SC Paderborn 07 HomeTeam, Def_form, I
done n
done o
done p
done q
done r
done s
done t
done extra
done a
done b
done c
done d
done e
done f
done g
done h
done i
done j
done k
done l
done m
done n
done o
done p
done q
done r
done s
done t
done extra
done a
done b
done c
done d
done e
done f
done g
done h
done i
done j
done k
done l
done m
done n
done o
done p
done q
done r
done s
done t
done extra
done a
done b
done c
done d
done e
done f
done g
done h
done i
done j
done k
done l
done m
done n
done o
done p
done q
done r
done s
done t
done extra
done a
done b
done c
done d
done e
done f
done g
done h
done i
done j
done k
done l
done m
done n
done o
done p
done q
done r
done s
done t
done extra
done a
done b
done c
done d
done e
done f
done g
done h
done i
done j
done k
done l
done m
done n
done o
done p
done q
done r
done s
done t


In [365]:
Husky_LL2.tail(10)

,League,Date,Time,HT,AT,H_xPts_Total,H_xPts_H,A_xPts_Total,A_xPts_A,xPoints_Diff_Total,xPoints_Diff_H-A,HT_HPoints_5,AT_APoints_5,HA_PointsDiff,HxG_5,Last_HxG,HxPTS_5,Last_HxPTS,HPass%_5,Last_HPass%,HPPDA_5,Last_HPPDA,HDEEP_5,Last_HDEEP,HCCC_5,Last_HCCC,HxConcd_5,Last_HxConcd,AxG_5,Last_AxG,AxPTS_5,Last_AxPTS,APass%_5,Last_APass%,APPDA_5,Last_APPDA,ADEEP_5,Last_ADEEP,ACCC_5,Last_ACCC,AxConcd_5,Last_AxConcd,xG_Diff_5,xPTS_Diff_5,Pass%_Diff_5,PPDA_Diff_5,DEEP_Diff_5,CCC_Diff_5,xConcd_Diff_5,SQuality_H5,SQuality_A5,SQuality_Diff5,HS_5,Last_H_S,HST_5,Last_H_ST,HScrd_5,Last_H_Scrd,HConcd_5,Last_H_Concd,AS_5,Last_A_S,AST_5,Last_A_ST,AScrd_5,Last_A_Scrd,AConcd_5,Last_A_Concd,S_Diff_H-A_5,ST_Diff_H-A_5,Scrd_Diff_H-A_5,Concd_Diff_H-A_5,WinCounterTotal_H,WinCounterTotal_A,WinCounter_H,WinCounter_A,ScrdCounterTotal_H,ScrdCounterTotal_A,ScrdCounter_H,ScrdCounter_A,ConcdCounterTotal_H,ConcdCounterTotal_A,ConcdCounter_H,ConcdCounter_A,Lucky_H5,Lucky_A5,Lucky_Diff5,Unlucky_H5,Unlucky_A5,Unlucky_Diff5,STForm_H5,STForm_A5,STForm_Diff5,CCCForm_H5,CCCForm_A5,CCCForm_Diff5,DEEPForm_H5,DEEPForm_A5,DEEPForm_Diff5,PPDAForm_H5,PPDAForm_A5,PPDAForm_Diff5,PassForm_H5,PassForm_A5,PassForm_Diff5,DeForm_H5,DeForm_A5,DeForm_Diff5,AtkForm_H5,AtkForm_A5,AtkForm_Diff5,Form1_H5,Form1_A5,Form1_Diff5,Form2_H5,Form2_A5,Form2_Diff5,Form3_H5,Form3_A5,Form3_Diff5,NewForm_H5,NewForm_A5,NewForm_Diff5,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
1850,French Ligue 1,2020-03-01,2000,Lyon,St Etienne,1.6,1.5,1.2,1.1,0.4,0.4,1,0,1,1,1,1.5,1,80,0,11,0,3,0,1,0,1,0,1,0,1.5,0,80,0,9,1,3,0,2,0,1,1,0,-0,1,2,0,0,0,0.3,0.3,0,10,1,4,1,1,1,1,0,12,0,4,0,1,1,2,0,-3,-1,0,-1,1,-5,-2,-6,3,2,1,1,1,-12,-1,-7,-2.4,-6.6,4.2,1,9,-8,-0,0,-0,1,0,1,-2,-1,-1,-1,-3,2,-1,1,-2,70,-5,75,-3,-19,16,1.3,1.5,-0.2,1.5,0.9,0.6,1.9,1.7,0.2,8.7,6.3,2.4,1,1.65,3.91,5.23
1851,French Ligue 1,2020-03-06,2000,Marseille,Amiens,1.5,1.7,1.1,1,0.4,0.7,2,0.4,1.6,1,1,1.2,0,81,0,9,0,4,0,1,1,1,1,1,1,0.8,1,71,1,9,0,4,1,1,1,1,0,0,0.4,10,0,0,1,0,0.3,0.2,0.1,10,1,3,1,2,1,1,1,8,1,2,1,1,0,1,0,2,1,1,0,1,-15,-1,-10,5,-2,2,-2,-3,-1,-1,2,6.1,-0.9,7,0.3,0.3,0,-1,-1,0,0,-0,0,-1,0,-1,-1,-4,3,0,-9,9,-40,2,-42,1,-58,59,0.9,0.4,0.5,1.5,1.2,0.3,1,0.7,0.3,6.4,3.8,2.6,0,1.57,3.99,6.13
1852,French Ligue 1,2020-03-07,1900,Metz,Nimes,1,1.1,1.2,0.8,-0.2,0.3,1.4,0.6,0.8,1,1,0.9,1,74,0,21,0,2,1,0,1,1,0,1,1,1.6,1,72,1,9,0,4,0,1,1,1,1,-1,-0.7,2,12,-1,-1,0,0.2,0.3,-0.1,11,1,4,1,1,1,1,0,11,1,4,1,2,1,1,1,0,0,-1,0,1,-2,-2,-1,1,7,-1,4,1,-2,-3,-7,0.3,0.9,-0.6,3.6,2,1.6,-1,0,-1,-1,0,-1,-3,0,-3,14,-2,16,-8,-7,-1,-47,-68,21,-95,38,-133,0.6,1.8,-1.2,0.5,1.1,-0.6,0.9,1.8,-0.9,2,6.6,-4.6,1,2.41,3.08,3.2
1853,French Ligue 1,2020-03-07,1900,Angers,Nantes,1.4,1.6,1.3,1.2,0.1,0.4,1,2,-1,1,0,1.2,0,78,0,18,1,4,0,1,0,1,1,1,0,1,0,76,0,16,0,3,0,1,0,2,1,0,0.2,2,3,1,-1,-1,0.2,0.3,-0.1,11,0,3,0,0,1,1,0,11,0,4,0,1,0,1,0,0,-1,-1,-1,2,-1,1,1,2,-1,1,5,2,-2,1,-3,0,-0.2,0.2,4.4,3.1,1.3,-2,0,-2,-1,-0,-1,-3,-1,-2,9,-2,11,-5,-5,0,-2,-8,6,-74,-44,-30,0.8,0.8,0,0.8,1.4,-0.6,1.6,0.8,0.8,3.9,4.6,-0.7,1,2.38,3.06,3.28
1854,French Ligue 1,2020-03-07,1900,Reims,Brest,1.4,1.5,1.1,0.6,0.3,0.9,1.6,1,0.6,1,1,1.3,0,83,0,13,1,6,0,1,1,1,1,1,0,1.3,1,81,1,14,0,5,1,1,0,1,0,0,0,2,-1,1,0,0,0.3,0.3,0,12,1,3,1,1,1,1,0,12,1,3,1,1,0,1,0,0,0,0,0,-2,-2,1,-3,3,-1,2,1,-2,-3,1,-1,-0.6,-0.6,0,1.9,5.9,-4,1,-0,1,-0,-1,1,2,3,-1,0,4,-4,2,5,-3,3,2,1,1,-18,19,0.8,1.2,-0.4,1.7,1.4,0.3,1,1.1,-0.1,8.6,7,1.6,1,1.82,3.45,4.68
1855,French Ligue 1,2020-03-07,1900,Dijon,Toulouse,1.1,1.4,1,0.8,0.1,0.6,1.8,0.2,1.6,1,0,0.8,0,79,1,16,1,3,0,1,0,2,1,1,0,1.3,0,81,0,17,1,4,1,1,0,2,0,0,-0.4,-3,-1,0,0,1,0.3,0.5,-0.2,11,0,3,0,1,0,2,1,12,1,2,0,0,0,2,1,-2,1,1,1,-5,-17,-2,-14,-1,-7,9,-5,-5,-5,-2,-2,-1.2,-6.4,5.2,2.6,9.7,-7.1,-1,-1,0,-0,-0,0,-2,0,-2,6,7,-1,2,2,0,-66,-55,-11,-38,19,-57,0.7,1.4,-0.7,0.9,1.3,-0.4,0.6,1.1,-0.5,3.7,5.8,-2.1,1,1.87,3.58,4.24
1856,French Ligue 1,2020-03-07,1900,Nice,Monaco,1.3,1.5,1.5,1.2,-0.2,0.3,1.6,1.2,0.4,1,0,1.1,0,88,0,14,0,4,0,2,0,2,1,2,1,1.7,1,82,1,8,0,9,1,1,0,1,1,0,-0.7,6,5,-5,1,1,0.3,0.3,0,11,0,4,0,1,0,

In [371]:
Husky_LL2.to_excel('Husky2_Bund.xlsx',index=False)

In [75]:
byodd_alpha('PremAlpha06_20')

In [64]:
PremAlpha15_20=league_alpha('Premier_League',2015,2020,5)
PremAlpha18_20=league_alpha('Premier_League',2018,2020,5)
PremAlpha20=league_alpha('Premier_League',2020,2020,5)

In [66]:
PremAlpha15_20.to_excel('PremAlpha15_20.xlsx',index=False)
PremAlpha18_20.to_excel('PremAlpha18_20.xlsx',index=False)
PremAlpha20.to_excel('PremAlpha20.xlsx',index=False)

In [67]:
byodd_alpha('PremAlpha15_20')
byodd_alpha('PremAlpha18_20')
byodd_alpha('PremAlpha20')

In [31]:
PremAlpha06_19=league_alpha('Premier_League',2006,2019,5)
PremAlpha06_18=league_alpha('Premier_League',2006,2018,5)
PremAlpha06_17=league_alpha('Premier_League',2006,2017,5)

In [32]:
PremAlpha06_19.to_excel('PremAlpha06_19.xlsx',index=False)
PremAlpha06_18.to_excel('PremAlpha06_18.xlsx',index=False)
PremAlpha06_17.to_excel('PremAlpha06_17.xlsx',index=False)

In [33]:
byodd_alpha('PremAlpha06_19')
byodd_alpha('PremAlpha06_18')
byodd_alpha('PremAlpha06_17')

In [34]:
PremAlpha18=league_alpha('Premier_League',2018,2018,5)
PremAlpha19=league_alpha('Premier_League',2019,2019,5)

In [35]:
PremAlpha18.to_excel('PremAlpha18.xlsx',index=False)
PremAlpha19.to_excel('PremAlpha19.xlsx',index=False)

In [36]:
byodd_alpha('PremAlpha18')
byodd_alpha('PremAlpha19')